In [27]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, when
from pyspark.sql.functions import row_number, desc
import pyspark.sql.functions as sf

from datetime import datetime
import os

In [2]:
spark = SparkSession.builder.config("spark.driver.memory", "8g").config("spark.executor.cores",8).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/05 11:04:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [62]:
def etl_process(path, file_name, file_date):
    spark = SparkSession.builder.config("spark.driver.memory", "8g").config("spark.executor.cores",8).getOrCreate()
    
    
    df = spark.read.json(path+file_name)
    df = df.select('_source.*')
    
    df = df.withColumn("Type",
           when((col("AppName") == 'CHANNEL') | (col("AppName") =='DSHD')| (col("AppName") =='KPLUS')| (col("AppName") =='KPlus'), "Truyền Hình")
        .when((col("AppName") == 'VOD') | (col("AppName") =='FIMS_RES')| (col("AppName") =='BHD_RES')| 
              (col("AppName") =='VOD_RES')| (col("AppName") =='FIMS')| (col("AppName") =='BHD')| (col("AppName") =='DANET'), "Phim Truyện")
        .when((col("AppName") == 'RELAX'), "Giải Trí")
        .when((col("AppName") == 'CHILD'), "Thiếu Nhi")
        .when((col("AppName") == 'SPORT'), "Thể Thao")
        .otherwise("Error"))
    
    df = df.withColumn("Date", sf.lit(file_date))
    df = df.select('Contract','Type','TotalDuration', 'Date')
    df = df.filter(df.Type != 'Error')
    
    print('Finished Processing {}'.format(file_name))
    return df

In [63]:
path = '/Users/habaokhanh/Study_BigData_Dataset/log_content/'
list_file = sorted([file for file in os.listdir(path) if file != '.DS_Store'])

In [64]:
start_date_str = '20220401'
end_date_str = '20220430'

In [65]:
file_name = "20220401.json"
date_str = file_name.split('_')[-1].split('.')[0]
file_date = datetime.strptime(date_str, "%Y%m%d").date()

In [66]:
test = etl_process(path, file_name, file_date)

Finished Processing 20220401.json


In [74]:
test.show()

+---------+-----------+-------------+----------+
| Contract|       Type|TotalDuration|      Date|
+---------+-----------+-------------+----------+
|HNH579912|Truyền Hình|          254|2022-04-01|
|HUFD40665|Truyền Hình|         1457|2022-04-01|
|HNH572635|Truyền Hình|         2318|2022-04-01|
|HND141717|Truyền Hình|         1452|2022-04-01|
|HNH743103|Truyền Hình|          251|2022-04-01|
|HNH893773|Truyền Hình|          924|2022-04-01|
|HND083642|Truyền Hình|         1444|2022-04-01|
|DNFD74404|Truyền Hình|          691|2022-04-01|
|DTFD21200|Truyền Hình|         1436|2022-04-01|
|LDFD05747|Truyền Hình|         1434|2022-04-01|
|HNH063566|Truyền Hình|          687|2022-04-01|
|HNH866786|Truyền Hình|          248|2022-04-01|
|NBAAA1128|Truyền Hình|          247|2022-04-01|
|HNH960439|Truyền Hình|          683|2022-04-01|
|HNJ035736|Truyền Hình|          246|2022-04-01|
|NTFD93673|Truyền Hình|         2288|2022-04-01|
|HNJ063267|Truyền Hình|         2282|2022-04-01|
|HNH790383|Truyền Hì

In [68]:
result = None
for file_name in list_file:
    date_str = file_name.split('_')[-1].split('.')[0]
    file_date = datetime.strptime(date_str, "%Y%m%d").date()
    
    start_date = datetime.strptime(start_date_str, "%Y%m%d").date()
    end_date = datetime.strptime(end_date_str, "%Y%m%d").date()
    if start_date <= file_date <= end_date:
        df = etl_process(path, file_name, file_date)
        if result is None:
            result = df
        else:
            result = result.union(df)
            
result = result.groupby('Contract','Type','Date').sum()
result = result.withColumnRenamed('sum(TotalDuration)','TotalDuration')
final = result.groupBy('Contract','Date').pivot("Type").sum('TotalDuration')
final = final.withColumnRenamed('Giải Trí', 'RelaxDuration') \
    .withColumnRenamed('Phim Truyện', 'MovieDuration') \
    .withColumnRenamed('Thiếu Nhi', 'ChildDuration') \
    .withColumnRenamed('Thể Thao', 'SportDuration') \
    .withColumnRenamed('Truyền Hình', 'TVDuration')

Finished Processing 20220401.json
Finished Processing 20220402.json
Finished Processing 20220403.json
Finished Processing 20220404.json
Finished Processing 20220405.json
Finished Processing 20220406.json
Finished Processing 20220407.json
Finished Processing 20220408.json
Finished Processing 20220409.json
Finished Processing 20220410.json
Finished Processing 20220411.json
Finished Processing 20220412.json
Finished Processing 20220413.json
Finished Processing 20220414.json
Finished Processing 20220415.json
Finished Processing 20220416.json
Finished Processing 20220417.json
Finished Processing 20220418.json
Finished Processing 20220419.json
Finished Processing 20220420.json
Finished Processing 20220421.json
Finished Processing 20220422.json
Finished Processing 20220423.json
Finished Processing 20220424.json
Finished Processing 20220425.json
Finished Processing 20220426.json
Finished Processing 20220427.json
Finished Processing 20220428.json
Finished Processing 20220429.json
Finished Proce

In [72]:
final.select(col('Date')).distinct().show(30)

+----------+
|      Date|
+----------+
|2022-04-01|
|2022-04-02|
|2022-04-03|
|2022-04-04|
|2022-04-05|
|2022-04-06|
|2022-04-07|
|2022-04-08|
|2022-04-09|
|2022-04-10|
|2022-04-11|
|2022-04-12|
|2022-04-13|
|2022-04-14|
|2022-04-15|
|2022-04-16|
|2022-04-17|
|2022-04-18|
|2022-04-19|
|2022-04-20|
|2022-04-21|
|2022-04-22|
|2022-04-23|
|2022-04-24|
|2022-04-25|
|2022-04-26|
|2022-04-27|
|2022-04-28|
|2022-04-29|
|2022-04-30|
+----------+



calculate most_watch

In [12]:
result.show(10)

+---------+-----------+-------------+
| Contract|       Type|TotalDuration|
+---------+-----------+-------------+
|SGH292167|Phim Truyện|       298535|
|NTFD16511|Truyền Hình|       376444|
|TNFD11777|Truyền Hình|       184403|
|LCFD14656|Truyền Hình|        55695|
|HNJ070566|Truyền Hình|       869801|
|HNH259245|Truyền Hình|       760514|
|VTD034451|Truyền Hình|       290270|
|QNFD66748|Truyền Hình|       355154|
|QND055814|Truyền Hình|       511338|
|HBFD09242|Truyền Hình|       261433|
+---------+-----------+-------------+
only showing top 10 rows



In [20]:
windowSpec = Window.partitionBy("Contract").orderBy(desc("TotalDuration"))
mostWatch = result.withColumn("rank",row_number().over(windowSpec))
mostWatch = mostWatch.filter(mostWatch.rank==1)
mostWatch = mostWatch.select("Contract", "Type")
mostWatch = mostWatch.withColumnRenamed("Type","MostWatch")
mostWatch.show()

+--------------+-----------+
|      Contract|  MostWatch|
+--------------+-----------+
|113.182.209.48|   Giải Trí|
|14.182.110.125|Truyền Hình|
|14.189.124.168|Truyền Hình|
|     AGAAA0338|Truyền Hình|
|     AGAAA0342|Truyền Hình|
|     AGAAA0346|Truyền Hình|
|     AGAAA0353|Truyền Hình|
|     AGAAA0372|Truyền Hình|
|     AGAAA0391|Truyền Hình|
|     AGAAA0452|Truyền Hình|
|     AGAAA0504|Truyền Hình|
|     AGAAA0544|Truyền Hình|
|     AGAAA0550|Truyền Hình|
|     AGAAA0555|Truyền Hình|
|     AGAAA0576|Truyền Hình|
|     AGAAA0593|Truyền Hình|
|     AGAAA0608|Truyền Hình|
|     AGAAA0613|Truyền Hình|
|     AGAAA0638|Truyền Hình|
|     AGAAA0663|Truyền Hình|
+--------------+-----------+
only showing top 20 rows



In [21]:
def most_watch_calc(result):
    windowSpec = Window.partitionBy("Contract").orderBy(desc("TotalDuration"))
    mostWatch = result.withColumn("rank",row_number().over(windowSpec))
    mostWatch = mostWatch.filter(mostWatch.rank==1)
    mostWatch = mostWatch.select("Contract", "Type")
    mostWatch = mostWatch.withColumnRenamed("Type","MostWatch")
    return mostWatch

calculate customer taste

In [23]:
final.printSchema()

root
 |-- Contract: string (nullable = true)
 |-- RelaxDuration: long (nullable = true)
 |-- MovieDuration: long (nullable = true)
 |-- ChildDuration: long (nullable = true)
 |-- SportDuration: long (nullable = true)
 |-- TVDuration: long (nullable = true)



In [30]:
def customer_tase_calc(final):
    final = final.withColumn("RelaxDuration",when(col("RelaxDuration").isNotNull(),"Relax").otherwise(col("RelaxDuration")))
    final = final.withColumn("MovieDuration",when(col("MovieDuration").isNotNull(),"Movie").otherwise(col("MovieDuration")))
    final = final.withColumn("ChildDuration",when(col("ChildDuration").isNotNull(),"Child").otherwise(col("ChildDuration")))
    final = final.withColumn("SportDuration",when(col("SportDuration").isNotNull(),"Sport").otherwise(col("SportDuration")))
    final = final.withColumn("TVDuration",when(col("TVDuration").isNotNull(),"TV").otherwise(col("TVDuration")))

    taste = final.withColumn('CustomerTaste', sf.concat_ws("-", *[i for i in final.columns if i != 'Contract']))
    return taste

calculate activeness

Transforming 1 days of data
